In [1]:
# 1. Import necessary libraries
from ultralytics import YOLO # Here we import YOLO
import yaml                  # for yaml files
import torch
from PIL import Image
import os
import cv2
import time

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")

In [3]:
# 2. Choose our yaml file
yaml_filename = 'gun_detection.yaml'

In [4]:
!wandb login

In [5]:
import os
import shutil
import random

def create_dirs(base_path):
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    for dir_name in ['train', 'val', 'test']:
        os.makedirs(os.path.join(base_path, dir_name, 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_path, dir_name, 'labels'), exist_ok=True)

def copy_files(src, dst, files, prefix):
    for f in files:
        src_file = os.path.join(src, f)
        dst_file = os.path.join(dst, f"{prefix}_{f}")
        print(f"Copiando {f} da {src} a {dst_file}")
        shutil.copy(src_file, dst_file)

def reduce_dataset(base_path, reduced_base_path, split_ratios=(0.7, 0.2, 0.1), reduction_factor=0.1):
    print(f"Proporzioni fornite: {split_ratios}")
    print(f"Somma delle proporzioni: {sum(split_ratios)}")
    assert round(sum(split_ratios), 10) == 1, "Le proporzioni devono sommare a 1."
    create_dirs(reduced_base_path)

    categories = ['Handgun', 'Machine_Gun', 'No_Gun']
    for category in categories:
        category_path = os.path.join(base_path, category)
        subfolders = [f for f in os.listdir(category_path) if os.path.isdir(os.path.join(category_path, f))]
        print(f"Categoria: {category}, Sottocartelle: {subfolders}")
        random.shuffle(subfolders)

        reduced_subfolders = subfolders[:int(len(subfolders) * reduction_factor)]

        train_split = int(split_ratios[0] * len(reduced_subfolders))
        val_split = int(split_ratios[1] * len(reduced_subfolders))
        train_subfolders = reduced_subfolders[:train_split]
        val_subfolders = reduced_subfolders[train_split:train_split + val_split]
        test_subfolders = reduced_subfolders[train_split + val_split:]

        for split, split_subfolders in zip(['train', 'val', 'test'], [train_subfolders, val_subfolders, test_subfolders]):
            for subfolder in split_subfolders:
                frame_path = os.path.join(category_path, subfolder, 'frames')
                if not os.path.exists(frame_path):
                    continue
                images = [f for f in os.listdir(frame_path) if f.endswith('.jpg')]
                labels = [f for f in os.listdir(frame_path) if f.endswith('.txt')]

                print(f"Split: {split}, Subfolder: {subfolder}, Images: {images}, Labels: {labels}")

                copy_files(frame_path, os.path.join(reduced_base_path, split, 'images'), images, f"{category}_{subfolder}")
                copy_files(frame_path, os.path.join(reduced_base_path, split, 'labels'), labels, f"{category}_{subfolder}")

# Esegui la riduzione del dataset
reduce_dataset(
    base_path='./Gun_Action_Recognition_Dataset',
    reduced_base_path='./Reduced_Gun_Action_Recognition_Dataset',
    split_ratios=(0.7, 0.2, 0.1),
    reduction_factor=0.1  # Usa il 10% del dataset originale
)


In [6]:
# 3. Create Yolo model
model = YOLO('yolov8n.yaml') # creates Yolo object from 'yolov8n.yaml' configuration file.
model = YOLO('Reduced_Gun_Action_Recognition_Dataset/yolov8n.pt')   # Loads pretrained weights
model = YOLO('yolov8n.yaml').load('Reduced_Gun_Action_Recognition_Dataset/yolov8n.pt')  # build from YAML and transfer weights

In [7]:
data_config = """
train: /Users/andreavisi/Desktop/PYTHON/Computer Vision e Deep Learning 2024/PROGETTO/Reduced_Gun_Action_Recognition_Dataset/train/images
val: /Users/andreavisi/Desktop/PYTHON/Computer Vision e Deep Learning 2024/PROGETTO/Reduced_Gun_Action_Recognition_Dataset/val/images

# Numero di classi
nc: 2

# Nomi delle classi
names: ['Handgun', 'Machine_Gun']
"""

with open('gun_detection.yaml', 'w') as f:
    f.write(data_config)

In [ ]:
# 4. Train the model
model.train(data='{}'.format(yaml_filename), epochs=30,patience=5, batch=16,  imgsz=640)